# Import libraries

In [4]:
from warnings import simplefilter # import warnings filter
simplefilter(action='ignore', category=FutureWarning) # ignore all future warnings

from CD_tools import CDML, compare_binary_columns, df_2_xlsx, df_balance, high_correlation_filter, PCA_analyse, voting_classifier, zero_variance_columns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold

ImportError: DLL load failed: The specified module could not be found.

# Options

In [ ]:
%matplotlib inline
pd.options.display.max_columns = None
plt.rcParams['figure.figsize'] = [16, 8]

# Read and prepare the house dataset

In [ ]:
df_house = pd.read_csv('D:/Documents/4. Science/Data Science/Datasets/House sales/kc_house_data.csv',header=0)
df_house['sqft_living'] = np.asarray(df_house['sqft_living'],dtype=np.float64) # Convert 'sqft_living' from integer to float to prevent errors when running models.
df_house = df_house[df_house['bedrooms'] != 33] # Remove house with 33(!) bedrooms.
df_house.sort_values(by=['sqft_living','price'])

keys = ['id','date']
features = ['sqft_living','sqft_lot','sqft_above','sqft_basement']

features = ['bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement',
       'sqft_living15', 'sqft_lot15']

#features = ['x1','x2']
#features_categorical = []

features_categorical = ['bedrooms', 'bathrooms',
                       'floors', 'waterfront', 'view', 'condition', 'grade']

features_categorical_classes_2_drop = []
label = ['price']
#label = ['y']

#features = ['sqft_living','sqft_lot','view','sqft_above','sqft_basement']
#features_categorical = ['view']
#features_categorical_classes_2_drop = [4]

#df_house['x1'] = df_house['price']
#df_house['x2'] = df_house['price'].apply(lambda x: x**2)
#df_house['y'] = df_house['price'].apply(lambda x: x**2)
#df_house = df_house[keys+features+label]

In [ ]:
df_house.columns

# Build a regression model with default settings

In [ ]:
rc = CDML(df_house,keys,features,label[0],features_categorical=features_categorical,features_categorical_classes_2_drop=features_categorical_classes_2_drop)
rc.split_data(test_size=0.2,random_state=None,sampling=None)
#rc.regression_model_data("LinearRegression()",nrfolds=5)
rc.regression_model_data("RandomForestRegressor()",nrfolds=5)
rc.regression_save_weights()
maximum = 1.01*max(rc.data.df[rc.data.label[0]])
rc.regression_show_residuals(xmin=0,xmax=maximum,ymin=0,ymax=maximum,nrbins=201)
rc.show_learning_curve(ylim=None,cv=None,n_jobs=4,train_sizes=np.linspace(0.3,1.0,10),scoring='r2')
rc.save_feature_importance()

In [ ]:
rc.model.df_train_incl_predictions

In [ ]:
rc.model.df_test_incl_predictions

# Optimize the classification model using hyperparameter tuning

In [ ]:
# Setup hyperparameter search grid
random_grid = {'n_estimators': [10,50,100,150,200,250,500],
               'criterion': ['gini','entropy'],
               'max_depth': [5,10,None],
               'min_samples_split': [2,10,50,100,500],
               'min_samples_leaf': [1,10,50,100,500],
               'min_weight_fraction_leaf': [0.0],
               'max_features': ['auto',None],
               'max_leaf_nodes': [None],
               'min_impurity_decrease': [0.0],
               'min_impurity_split': [None],
               'bootstrap': [True],
               'oob_score': [False],
               'n_jobs': [-1],
               'random_state': [0],
               'verbose': [0],
               'warm_start': [False],
               'class_weight': [None]
              }

# Search the hyperparameter grid for the optimal hyperparameters
rc = CDML(df_titanic,keys,features,features_categorical,label[0],features_categorical_classes_2_drop=features_categorical_classes_2_drop)
rc.split_data(test_size=0.1,random_state=0,sampling=None)
rc.classification_model_data("RandomForestClassifier(random_state=0)",threshold=0.5) # Run a model with default settings
rc.RandomizedSearchCV(random_grid,n_iter=100)
best_estimator = rc.model.randomized_search_CV.random_search.best_estimator_
print("Best estimator:")
print(best_estimator)
print()
# Run the model with the optimal hyperparameters
#rc = CDML(df_titanic,keys,features,features_categorical,label[0],features_categorical_classes_2_drop=features_categorical_classes_2_drop)
#rc.split_data(test_size=0.1,random_state=0,sampling=None)
rc.classification_model_data(str(best_estimator),threshold=0.5) # Run the model with the optimal hyperparameters
rc.show_learning_curve(ylim=(0,1),cv=StratifiedKFold(12),n_jobs=4,train_sizes=np.linspace(0.3,1.0,10),scoring='roc_auc')
rc.classification_show_ROC_precision_recall_curves(show_labels='Y',label_interval=6,label_offsets_ROC=[15,-20],label_offsets_PR=[-30,-30])
rc.classification_show_prediction_distributions(nrbins=51)
rc.save_feature_importance()
rc.classification_show_interpretation_table(sort_columns='Y',top=5)
rc.classification_transpose_interpretation_table()
rc.classification_show_interpretation_table_LIME(top=5,num_samples=5000)
rc.classification_transpose_interpretation_table_LIME()